# Graph RAG
- 提取关键词
- 查询图数据库检索上下文
- 利用上下文生成答案

In [3]:
from model import RagLLM, QwenLLM
from langchain import PromptTemplate

## 解析查询，提取关键词

In [4]:
kg_llm = RagLLM()

In [5]:
prompt = """给定一些初始查询，提取最多 {max_keywords} 个相关关键词，考虑大小写、复数形式、常见表达等。
用 '^' 符号分隔所有同义词/关键词：'关键词1^关键词2^...'

注意，结果应为一行，用 '^' 符号分隔。
----
查询: {query_str}
----
关键词:
"""

In [6]:
prompt_template = PromptTemplate(
                input_variables=["query_str", "max_keywords"],
                template=prompt,
            )

In [7]:
final_prompt = prompt_template.format(max_keywords='3',
                             query_str="格力电器有什么事情发生")

In [8]:
final_prompt

"给定一些初始查询，提取最多 3 个相关关键词，考虑大小写、复数形式、常见表达等。\n用 '^' 符号分隔所有同义词/关键词：'关键词1^关键词2^...'\n\n注意，结果应为一行，用 '^' 符号分隔。\n----\n查询: 格力电器有什么事情发生\n----\n关键词:\n"

In [9]:
kg_llm(final_prompt)

'格力电器^事件^新闻'

In [10]:
def parse_query(query, max_keywords=3):
    final_prompt = prompt_template.format(max_keywords='3',
                                          query_str=query)
    response = kg_llm(final_prompt)
    keywords = response.split('\n')[0].split('^')
    return keywords

In [11]:
parse_query("总结下哪些公司进行了企业转型？")

['公司', '企业转型', '总结']

## Graph RAG

### 通过关键词来查询节点

In [12]:
from py2neo import Graph, Node, Relationship

In [13]:
graph = Graph("bolt://180.101.26.237:7687", user='neo4j', password='neo4j@123',name='neo4j')

In [14]:
def get_node(keyword, node_type):
    query = f"""
    MATCH (n:{node_type})
    where n.name CONTAINS "{keyword}"
    RETURN n.name as name
    """
    results = graph.run(query)
    print(query)
    for record in results:
        return record['name']
    return None

In [16]:
res = get_node("比亚迪", "EventType")
print(res)


    MATCH (n:EventType)
    where n.name CONTAINS "比亚迪"
    RETURN n.name as name
    
None


In [17]:
res = get_node("比亚迪", "Company")
print(res)


    MATCH (n:Company)
    where n.name CONTAINS "比亚迪"
    RETURN n.name as name
    
比亚迪股份有限公司


In [18]:
res = get_node("比亚迪", "Investor")
print(res)


    MATCH (n:Investor)
    where n.name CONTAINS "比亚迪"
    RETURN n.name as name
    
None


### 通过节点来查询关联的关系和节点信息

In [51]:
def gen_contexts(investor_condition, 
                 company_condition, 
                 even_type_condition,
                 query_level=1,
                 exclude_content=False):
    if query_level == 1:
        query = f"""
        MATCH (i:Investor)-[:INVEST]->(c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1 {investor_condition} {company_condition} {even_type_condition}
        RETURN i.name as investor,  c.name as company_name, e.name as even_type, r as relation
        """
    else:
        query = f"""
        MATCH (c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1 {investor_condition} {company_condition} {even_type_condition}
        RETURN c.name as company_name, e.name as even_type, r as relation
        """
    print(query)
    results = graph.run(query)
    contexts = []
    for record in results:
        context = ''
        record = dict(record)
        if 'investor' in record:
            context += f"{record['investor']} 投资了 {record['company_name']} \n"
        context = context + f"{record['company_name']} 发生了 {record['even_type']} \n 详细如下："
        for key, value in dict(record['relation']).items():
            if exclude_content:
                if key in ["title", "content"]:
                    continue
            context = context + f"\n  {key}: {value}"

        contexts.append(context)
    return contexts

In [20]:
def get_even_detail(keyword, exclude_content=False):
    investor = get_node(keyword, "Investor")
    company = get_node(keyword, "Company")
    even_type = get_node(keyword, "EventType")
    
    investor_condition = ""
    company_condition = ""
    even_type_condition = ""
    if investor:
        investor_condition = f' and i.name = "{investor}"'
    if company:
        company_condition = f' and c.name = "{company}"'
    if even_type:
        even_type_condition = f' and e.name = "{even_type}"'
        
    print(f"investor={investor_condition} company={company_condition} even_type={even_type_condition}")
    if investor_condition or company_condition or even_type_condition:
        contexts = gen_contexts(investor_condition, 
                                company_condition, 
                                even_type_condition,
                                query_level=1,
                                exclude_content=exclude_content)
        if len(contexts) == 0:
            contexts = gen_contexts(investor_condition, 
                                    company_condition, 
                                    even_type_condition,
                                    query_level=2,
                                    exclude_content=exclude_content)
        return contexts
    else:
        return []
        

In [21]:
contexts = get_even_detail(keyword="瑞幸咖啡", exclude_content=True)


    MATCH (n:Investor)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    
investor= company= and c.name = "瑞幸咖啡(北京)有限公司" even_type=

        MATCH (i:Investor)-[:INVEST]->(c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1   and c.name = "瑞幸咖啡(北京)有限公司" 
        RETURN i.name as investor,  c.name as company_name, e.name as even_type, r as relation
        

        MATCH (c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1   and c.name = "瑞幸咖啡(北京)有限公司" 
        RETURN c.name as company_name, e.name as even_type, r as relation
        


In [22]:
for c in contexts:
    print(c)
    print("="*50)

瑞幸咖啡(北京)有限公司 发生了 内部丑闻 
 详细如下：

  时间: 4月2日
瑞幸咖啡(北京)有限公司 发生了 停牌 
 详细如下：

  时间: 4月7日
瑞幸咖啡(北京)有限公司 发生了 股价异常波动 
 详细如下：

  数值: ['股价下跌了近89%']
  时间: 4月7日
瑞幸咖啡(北京)有限公司 发生了 造假欺诈 
 详细如下：

  数值: ['22亿元']
  时间: 4月2日
瑞幸咖啡(北京)有限公司 发生了 股权质押 
 详细如下：

  数值: ['4.88亿股', '49.13%', '1.45亿股', '30%']
  时间: 本年1月
瑞幸咖啡(北京)有限公司 发生了 自查违规 
 详细如下：

  时间: 7月1日晚间
瑞幸咖啡(北京)有限公司 发生了 复牌 
 详细如下：

  时间: 5月20日
瑞幸咖啡(北京)有限公司 发生了 资产被查封 
 详细如下：

瑞幸咖啡(北京)有限公司 发生了 董监高—主动离职 
 详细如下：

  时间: 4月23日
瑞幸咖啡(北京)有限公司 发生了 退市 
 详细如下：

  时间: 5月19日
瑞幸咖啡(北京)有限公司 发生了 董监高—被迫离职 
 详细如下：

  时间: 7月2日
瑞幸咖啡(北京)有限公司 发生了 收到起诉 
 详细如下：

  时间: 4月2日
瑞幸咖啡(北京)有限公司 发生了 其他违规行为 
 详细如下：

瑞幸咖啡(北京)有限公司 发生了 延期披露 
 详细如下：

  时间: 4月29日
瑞幸咖啡(北京)有限公司 发生了 产品服务负面消息 
 详细如下：



## graph rag pipline

In [24]:
rag_llm = RagLLM()

In [50]:
finance_template = """
你是金融知识助手，熟悉各种金融事件，需要根据提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接先回答不知道,要求用中文输出 \
问题：{question} 
上下文信息：
"{context}"
回答：
"""

In [48]:
def graph_rag_pipline(query, exclude_content=True, stream=True, temperature=0.1):
    keywords = parse_query(query, max_keywords=3)
    contexts = []
    ignore_words = ['公司', '分析', '投资']
    for keyword in keywords:
        if keyword in ignore_words:
            continue
        contexts.extend(get_even_detail(keyword=keyword, exclude_content=exclude_content))
    
    prompt = PromptTemplate(
                        input_variables=["question", "context"],
                        template=finance_template,)
    llm_prompt = prompt.format(question=query, context="\n========================\n".join(contexts))
    print(llm_prompt)
    
    if stream:
        response = rag_llm(llm_prompt, stream=True)
        print(f"response: ")
        for chunk in response:
            print(chunk.choices[0].text, end='', flush=True)
        return ""
    else:
        response = rag_llm(llm_prompt, stream=False, temperature=temperature)
        return response

In [57]:
query = "总结下哪些公司进行了企业转型？"

graph_rag_pipline(query, exclude_content=True)


    MATCH (n:Investor)
    where n.name CONTAINS "企业转型"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "企业转型"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "企业转型"
    RETURN n.name as name
    
investor= company= even_type= and e.name = "企业转型"

        MATCH (i:Investor)-[:INVEST]->(c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1    and e.name = "企业转型"
        RETURN i.name as investor,  c.name as company_name, e.name as even_type, r as relation
        

    MATCH (n:Investor)
    where n.name CONTAINS "总结"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "总结"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "总结"
    RETURN n.name as name
    
investor= company= even_type=

你是金融知识助手，熟悉各种金融事件，需要根据提供的上下文信息context来回答员工的提问。请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接先回答不知道,要求用中文输出 问题：总结下哪些公司进行了企业转型？ 
上下文信息：
"盈科资本 投资了 东易日盛家居装饰集团股份有限公司 
东易日盛家居装饰集团股份有限公司 发生了 企业转型 
 详细如下：

''

In [58]:
query = "分析下瑞幸咖啡？"

graph_rag_pipline(query, exclude_content=True)


    MATCH (n:Investor)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "瑞幸咖啡"
    RETURN n.name as name
    
investor= company= and c.name = "瑞幸咖啡(北京)有限公司" even_type=

        MATCH (i:Investor)-[:INVEST]->(c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1   and c.name = "瑞幸咖啡(北京)有限公司" 
        RETURN i.name as investor,  c.name as company_name, e.name as even_type, r as relation
        

        MATCH (c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1   and c.name = "瑞幸咖啡(北京)有限公司" 
        RETURN c.name as company_name, e.name as even_type, r as relation
        

    MATCH (n:Investor)
    where n.name CONTAINS "咖啡行业"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "咖啡行业"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "咖啡行业"
    RETURN n.name as name
    
investor= company= 

''

In [59]:
query = "中瑞深圳有没有投资什么公司？这些公司的状况怎么样"

graph_rag_pipline(query, exclude_content=True)


    MATCH (n:Investor)
    where n.name CONTAINS "中瑞深圳"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "中瑞深圳"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "中瑞深圳"
    RETURN n.name as name
    
investor= and i.name = "中瑞深圳" company= even_type=

        MATCH (i:Investor)-[:INVEST]->(c:Company)-[r:HAPPEN]->(e)
        WHERE 1=1  and i.name = "中瑞深圳"  
        RETURN i.name as investor,  c.name as company_name, e.name as even_type, r as relation
        

    MATCH (n:Investor)
    where n.name CONTAINS "公司状况"
    RETURN n.name as name
    

    MATCH (n:Company)
    where n.name CONTAINS "公司状况"
    RETURN n.name as name
    

    MATCH (n:EventType)
    where n.name CONTAINS "公司状况"
    RETURN n.name as name
    
investor= company= even_type=

你是金融知识助手，熟悉各种金融事件，需要根据提供的上下文信息context来回答员工的提问。请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接先回答不知道,要求用中文输出 问题：中瑞深圳有没有投资什么公司？这些公司的状况怎么样 
上下文信息：
"中瑞深圳 投资了 科大讯飞股份有限公司 
科大讯飞股份有限公司 发生了 裁员 
 详细如下

''